## 1.트레이너 데이터 생성

In [2]:
import cv2
import csv
import os
import mediapipe as mp
import timeit
import jk_module
import jk_vector
import jk_angle
import time

## Trainer 경로
raw_trainer_root_dir = 'D:/bagel/action_project2/data/'
trainer_video_name = 'short_trainer.mp4'
trainer_video_dir = os.path.join(raw_trainer_root_dir,trainer_video_name)

cap1 = cv2.VideoCapture(trainer_video_dir)

width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cv2.CAP_PROP_FPS

# VideoWriter 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'XVID')
trainer_output_video_name = 'trainer_keypoints.mp4'
trainer_video_output_dir = os.path.join(raw_trainer_root_dir,trainer_output_video_name)

out = cv2.VideoWriter(trainer_video_output_dir, fourcc, 3, (width1 , height1))



print('width', width1)
print('height', height1)

# 미디어 파이프 모델 선택
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

row = 0
# main code
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    while True:
        ret1, frame1 = cap1.read()

        if not ret1:
            break
        frame_time=cap1.get(cv2.CAP_PROP_POS_MSEC)

        # 시작 시점 측정
        start_t = timeit.default_timer()
        
        try:
            image1, results1 = jk_module.mediapipe_detection(frame1, pose)
            mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=5, circle_radius=3), 
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=5, circle_radius=3)) 
            landmarks = results1.pose_world_landmarks.landmark
            ## 점, 벡터, 각도 추출
            main_keypoint_json = jk_module.extract_main_landmark(landmarks, frame_time)
            main_vector_json = jk_vector.extract_vector(main_keypoint_json)
            main_angle_json = jk_angle.extract_angle(main_vector_json)
            

            ## 저장
            csv_keypoints_path = 'trainer_keypoints.csv'
            csv_vectors_path = 'trainer_vectors.csv'
            csv_angle_path = 'trainer_angles.csv'
            
            jk_angle.write_json_to_csv(csv_keypoints_path, main_keypoint_json)
            jk_angle.write_json_to_csv(csv_vectors_path, main_vector_json)
            jk_angle.write_json_to_csv(csv_angle_path, main_angle_json)
            
            csv_results_path = f'trainer_results.csv'
            csv_world_results_path =  f'trainer_world_results.csv'
            jk_module.landmarks_save(csv_results_path, results1, row)
            jk_module.world_landmarks_save(csv_world_results_path, results1, row)

            cv2.putText(image1,f'FPS: {FPS:.2f}', (860,30), cv2.FONT_ITALIC, 1, (255,0,0), 2)
            cv2.putText(image1,f'frame: {row:.2f}', (30,60), cv2.FONT_ITALIC, 1, (255,0,0), 2)
            angle = main_angle_json['lknee_angle(25)']
            cv2.putText(image1,f'angle: {angle:.2f}', (30,90), cv2.FONT_ITALIC, 1, (255,0,0), 2)
                
        except:
            pass
        

        row += 1

        # 종료 시점 측정
        end_t = timeit.default_timer()
        FPS = 1./(end_t - start_t )
        
        
        out.write(image1)
        cv2.imshow('st', image1)
        #time.sleep(0.1)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap1.release()
    out.release()
    cv2.destroyAllWindows()

width 822
height 586
